In [ ]:
pip install pandas

In [ ]:
pip install openpyxl

In [114]:
import pandas as pd

In [115]:
# 파일명
xls = pd.ExcelFile('pencil_tree_verb_adj.xlsx')

# Daraframe형식으로 엑셀 파일 읽기
df = pd.read_excel(xls, engine='openpyxl')

In [116]:
from collections import defaultdict, Counter

freq_dict = defaultdict(dict)


for idx, row in df.iterrows():
    item, group = row["Item"], row["group"]
    vv, vv2, vv3 = row["VV"], row["VV2"], row["VV3"]
    va, va2 = row["VA"], row["VA2"]

    #초기화 코드
    if not freq_dict[item]:
        freq_dict[item]["Verb"] = {
            "Young": [],
            "old": []
        }
        freq_dict[item]["Adj"] = {
            "Young": [],
            "old": []
        }

    #작업 코드
    if vv and not isinstance(vv, float):
        freq_dict[item]["Verb"][group].append(vv)
    if vv2 and not isinstance(vv2, float):
        freq_dict[item]["Verb"][group].append(vv2)
    if vv3 and not isinstance(vv3, float):
        freq_dict[item]["Verb"][group].append(vv3)
    if va and not isinstance(va, float):
        freq_dict[item]["Adj"][group].append(va)
    if va2 and not isinstance(va2, float):
        freq_dict[item]["Adj"][group].append(va2)

# 카운터 객체 적용
for item, tag_dict in freq_dict.items():
    for tag, group_dict in tag_dict.items():
        for group, words in group_dict.items():
            freq_dict[item][tag][group] = Counter(words)

In [117]:
def create_df_from_counter(counter, key_tag):
    total = counter.total()
    words = [word for word, cnt in counter.most_common()]
    fre = [cnt for word, cnt in counter.most_common()]
    percent = [v / total * 100 for v in fre]

    return pd.DataFrame({
        key_tag: words,
        "fre": fre,
        "percent": percent
    })

# 가로 병합
def concat_horizontal(item, tag_dict):
    vy, vo, ay, ao = tag_dict["Verb"]["Young"], tag_dict["Verb"]["old"], tag_dict["Adj"]["Young"], tag_dict["Adj"]["old"]

    s1= pd.Series([item], name="Item")
    gn_s1 = pd.Series(["Young"], name="Group")
    gn_s2 = pd.Series(["old"], name="Group")

    df1 = create_df_from_counter(vy, "Verb")
    df2 = create_df_from_counter(vo, "Verb")
    df3 = create_df_from_counter(ay, "Adjective")
    df4 = create_df_from_counter(ao, "Adjective")

    result = pd.concat([s1, gn_s1, df1, gn_s2, df2, gn_s1, df3, gn_s2, df4], axis=1)

    return result

# 세로 병합
def concat_vertical(df_list):
    return pd.concat(df_list, ignore_index=True)


def process(freq_dict):
    df_list = [concat_horizontal(item, tag_dict) for item, tag_dict in freq_dict.items()]
    return concat_vertical(df_list)

In [118]:
result = process(freq_dict)

In [119]:
result

,Item,Group,Verb,fre,percent,Group,Verb,fre,percent,Group,Adjective,fre,percent,Group,Adjective,fre,percent
0,Scissors,Young,자르다,55.0,48.245614,old,자르다,36.0,33.962264,Young,날카롭다,2.0,40.0,old,좋다,1.0,33.333333
1,NaN,NaN,사용하다,10.0,8.771930,NaN,가지다,14.0,13.207547,NaN,없다,2.0,40.0,NaN,좋아하다,1.0,33.333333
2,NaN,NaN,있다,9.0,7.894737,NaN,오리다,8.0,7.547170,NaN,낫다,1.0,20.0,NaN,많다,1.0,33.333333
3,NaN,NaN,들다,5.0,4.385965,NaN,사용하다,5.0,4.716981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,오리다,5.0,4.385965,NaN,필요하다,4.0,3.773585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,NaN,NaN,NaN,NaN,NaN,NaN,올라가다,1.0,0.952381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1079,NaN,NaN,NaN,NaN,NaN,NaN,수고하다,1.0,0.952381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1080,NaN,NaN,NaN,NaN,NaN,NaN,뜨다2,1.0,0.952381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1081,NaN,NaN,NaN,NaN,NaN,NaN,필요하다,1.0,0.952381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
# 엑셀 생성
import os

dir_name = "freq_result"
file_name = "freq_result"

if not os.path.exists(dir_name):
    os.mkdir(dir_name)

pd.DataFrame(result).to_excel(f'{dir_name}/{file_name}.xlsx', index=False)